In [ ]:
# default_exp stockfish

# cheviz stockfish

This module integrates a chess engine, in this case the popular stockfish engine. The actual engine is provided by Python chess as [described here](https://python-chess.readthedocs.io/en/latest/engine.html). However, in our goal to build a complete ETL pipeline, we still need to get the engine running on our box. In a similar way as we got our data (PGN-encoded chess games), I'd like to use Jupyter Notebooks to perform the engine integration.

# 1. Motivation

For automated insights, we need a way to let the computer learn about our metrics. We need something to compare against our metrics, so I figured I start with stockfish, a very popular chess engine. Python chess also interfaces with stockfish so I won't need another dependency, either.

Stockfish is a small download and with the binaries having few dependencies they should be ready to go right after extracting the archive – no extra installation required. The engine should then help us by generating our training data.

# 2. Getting the engine

In [ ]:
!mkdir -p "tools"
!ls

00_core.ipynb	    cheviz	     index.ipynb  README.md	    tools
01_data.ipynb	    CONTRIBUTING.md  LICENSE	  requirements.txt
02_ui.ipynb	    data	     Makefile	  settings.ini
03_stockfish.ipynb  docs	     MANIFEST.in  setup.py


Assuming this notebook runs on a Linux box, we'll get the Linux binary [from the official website](https://stockfishchess.org/download/). We're not building high-end desktop software here (does such thing even exist?) so hard-coding the download URL for just the Linux version is OK. It might feel wrong, but the best advice to counter your (very valid) intuition is that we have to focus on our goal: automated insights. That is, don't spend time over-engineering the basic, non-data-sciency stuff in your pipeline *unless* your pipeline is ready to run in production and earn money for you. There is a reason why we use Python for all of this, so quick'n'dirty – to a certain degree – is the way to go.

We need to set a fake browser user-agent for our download request, otherwise we get 403'd. Normally that's a sign you're doing something wrong at extra costs to someone else (in this case ISP hosting fees for the stockfish communiy). At 1.7M of download size, which is probably smaller than a typical project's landing page these days, I don't feel guilty at all.

In [ ]:
#export
from pathlib import Path
import urllib.request
import shutil

def download():
    src = 'https://stockfishchess.org/files/stockfish-11-linux.zip'
    dst = Path().absolute() / 'tools' / 'stockfish.zip'
    
    if not dst.is_file():
        request = urllib.request.Request(src)
        request.add_header('Referer', 'https://stockfishchess.org/download/')
        request.add_header('User-Agent', 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)')
        
        with urllib.request.urlopen(request) as response, open(dst, 'wb') as dst_file:
            shutil.copyfileobj(response, dst_file)
            
    return dst

We still need to extract the downloaded Zip archive. Also, we need to make the stockfish binary executable. There are 3 binaries available in this version, we use what I guess is the default one. More quick'n'dirty hardcoding that I expect to break sooner than later.

In [ ]:
#export
from pathlib import Path
import zipfile
import os
import stat

def extract(dst:Path):
    if not dst.is_file():
        return
    
    with zipfile.ZipFile(dst, 'r') as zip_file:
        zip_file.extractall(Path().absolute() / 'tools')
        dst_extracted = Path().absolute() / 'tools' / zip_file.namelist()[0]
            
        # make binary executable
        dst_binary = dst_extracted / 'Linux' / 'stockfish_20011801_x64'
        st = os.stat(dst_binary)
        os.chmod(dst_binary, st.st_mode | stat.S_IEXEC)
        
        return dst_binary

In [ ]:
stockfish = extract(download())

Check if we can run stockfish now, but let's not get stuck in stockfish's command prompt. So we send 'quit' to it immediately, which will still return the version of the binary and its authors.

In [ ]:
!{stockfish} quit

Stockfish 11 64 by T. Romstad, M. Costalba, J. Kiiski, G. Linscott


The next step is to hook up the engine with Python chess, as described in their documentation: https://python-chess.readthedocs.io/en/latest/engine.html

PGN can be described as a hierarchical document model due to its ability to store variations next to the mainline. Python chess uses a recursive nodes structure to model PGN; `GameNode::add_main_variation(move: chess.Move)->GameNode` processes a move, adds it as a child to the current node and returns the child. If we only process the mainline moves, the structure effectively represents a linked list. We could also just use a chess.Board instance to replay the engine results back to Python chess. Since our data module handles PGN however, I thought I go the extra step here. The way engine, game and board interact feels fragile to me but that is perhaps because board isn't just a plain chess position viewer. Instead, it has game/engine logic of its own.

In [ ]:
#export
from pathlib import Path
import chess
import chess.engine
import chess.pgn
import cheviz.data
import datetime

def playGame(engine_path:Path, time_limit:float=0.1):
    engine = chess.engine.SimpleEngine.popen_uci(engine_path.as_posix())
    game = chess.pgn.Game()
    game.headers['Event'] = 'cheviz'
    game.headers['Date'] = datetime.date.today().strftime("%Y-%m-%d")
    game.headers['White'] = engine_path.name
    game.headers['Black'] = engine_path.name

    while not game.board().is_game_over():
        result = engine.play(game.board(), chess.engine.Limit(time=time_limit), info=chess.engine.Info.SCORE)
        game = game.add_main_variation(result.move)
        # result.info is sometimes empty, happens more reliably with very short time limits
        game.comment = result.info['score'] if 'score' in result.info else None

    engine.quit()
    # set game back to root node
    game = game.game()
    game.headers['Result'] = game.board().result
    
    return game

Computer chess is extremely technical and moves will pile up. Even with short time limits a chess engine vs chess engine game can take a couple seconds or even minutes. Too short of a time limit and computer chess turns into garbage. Notice that it's not the engine that's slow, it's how we interact with it and process results.

In [ ]:
%time game = playGame(stockfish)
display(game.mainline_moves())

CPU times: user 6.61 s, sys: 180 ms, total: 6.79 s
Wall time: 1min 1s


<Mainline at 0x7f3a2239d0b8 (1. e4 e5 2. Nc3 Nf6 3. Nf3 Bb4 4. Nxe5 O-O 5. Nf3 Re8 6. Be2 Bxc3 7. dxc3 Nxe4 8. O-O d6 9. Ne1 Bf5 10. Bf3 d5 11. Nd3 h6 12. Bxe4 dxe4 13. Nf4 Nd7 14. Qh5 Re5 15. Rd1 g5 16. Nd5 Nf6 17. Nxf6+ Qxf6 18. Be3 Qe6 19. h3 b6 20. c4 Re8 21. a4 Qg6 22. Qxg6+ Bxg6 23. a5 Rxa5 24. Rxa5 bxa5 25. Rd7 Rc8 26. g4 f5 27. Bd4 Be8 28. Rg7+ Kf8 29. Rh7 fxg4 30. hxg4 Rd8 31. Be3 Rb8 32. Bd4 Rd8 33. Be3 Kg8 34. Rxh6 Rd6 35. Bxg5 Rxh6 36. Bxh6 Ba4 37. Bf4 Bxc2 38. Bxc7 Bd1 39. Kh2 Bxg4 40. Kg3 Be2 41. c5 e3 42. fxe3 Kf7 43. Bxa5 Bb5 44. Kf4 Ke6 45. Ke4 Ba4 46. Kd4 Kd7 47. e4 Ke6 48. Be1 Be8 49. Ba5 Ba4 50. Bc7 Bc6 51. Ke3 Be8 52. Ba5 Bc6 53. Bc3 Be8 54. Kd4 Bd7 55. Be1 Bb5 56. Bd2 Bc6 57. Bc3 Bd7 58. Ke3 Bc6 59. Ba5 Be8 60. Bc7 a6 61. Ba5 Bd7 62. Kf4 Bb5 63. Ke3 Bd7 64. Bc7 Be8 65. Kd4 Bc6 66. Ba5 Ba8 67. Bb4 Bb7 68. Be1 Bc6 69. Bd2 Bb7 70. Bc3 Bc6 71. Bd2 Bb7 72. Bf4 Bc6 73. Bg3 Bb7 74. Be1 Bc6 75. Bc3 Bb5 76. Ba5 Bc6 77. Bc3 Ba8 78. Ke3 Bc6 79. Kf4 Bb7 80. Ke3 Bc6 81. Kf4 Bb

Let's look at the engine's game through our UI.

In [ ]:
import cheviz.ui

games_list = []
games_list.append(cheviz.data.from_game(game))
cheviz.ui.showGameUi(games_list)

interactive(children=(Dropdown(description='Metric:', options=(<function diffReduce at 0x7f3a290bae18>, <built…

We use the same display wrap trick as in the `02_ui.ipynb` notebook.

In [ ]:
from IPython.display import display, HTML

CSS = """
.output {
    flex-direction: row;
    flex-wrap: wrap;
}
"""

HTML('<style>{}</style>'.format(CSS))